# This script runs lmm, looking at the effect of BDI and BAI on recall performance 

 ## Let's start by creating a dataframe with all relevant information

In [59]:
from cmlreaders import CMLReader, get_data_index
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import scipy
from scipy import stats
pd.set_option('display.max_columns', None)
import pickle
from scipy import stats
import statsmodels.api as sm
from false_memory_functions import *

Load data from excel file

In [60]:
# BAI_high = 15
# BAI_low = 8
# BDI_high = 16
# BDI_low = 10

sel_sub = pd.DataFrame([])
master = pd.read_excel(r'./220302_all_neuropsych.xlsx',engine="openpyxl") #read master spreadsheet of all RAM subjects demo/neuropsych data

sub_mat=list(np.zeros(len(master))*np.nan)
df = get_data_index('r1')
# df_select = pd.DataFrame([])
for sub in df['subject'].unique():
    num = int(sub[2:5])
    for i in master.index:
        sel = master.iloc[i]['Subject Number']
        if sel == num:
            sub_mat[i]=sub
# df_select[0:50]
master['subject']=sub_mat

len(master)

601

In [61]:
# look at the output
master.head(4)

,Subject Number,Study site,Gender,Years of Education,Age of seizure onset,Age at Implant (calculated),Phase II Implant Date,Phase III Implant Date,Traumatic Brain Injury (TBI) Notes,Date of Pre-operative Report,BAI,BDI,PHQ-9,GAD-7,FSIQ4,FSIQ3,FSIQ2 (Four Subtest),FSIQ2 (Two Subtest),FSIQ1 (Four Subtest),FSIQ1 (Two Subtest),GAI,VCI,PRI,WMI,Pre-operative Report,Prior Neurosurgical Procedures (can select more than one),Prior Surgery: Resection Detail,Prior Surgery: Callosotomy Detail,Prior Surgery: Laser Ablation Detail,subject
0,1,University of Pennsylvania,Female,13.0,20.0,48.105026,2014-10-08,NaT,NaN,NaT,8.0,2.0,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,76.0,81,75,NaN,REP001_neurospych.pdf,NaN,NaN,NaN,NaN,R1001P
1,2,University of Pennsylvania,Female,16.0,34.0,49.383743,2014-11-12,NaT,NaN,NaT,9.0,0.0,NaN,NaN,86,NaN,NaN,NaN,NaN,NaN,84.0,89,84,NaN,R1002P_neuropsych.pdf,NaN,NaN,NaN,NaN,R1002P
2,3,University of Pennsylvania,Female,13.0,37.0,39.390382,2014-11-13,NaT,motor vehicle accident due to loss of consciou...,NaT,NaN,NaN,NaN,NaN,102,NaN,NaN,NaN,NaN,NaN,NaN,107,100,NaN,R1003P_neuropsych.pdf,NaN,NaN,NaN,NaN,R1003P
3,4,Dartmouth University,Female,12.0,6.0,51.918931,2014-11-06,NaT,"In early 90s, motor vehicle accident resulting...",NaT,NaN,NaN,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,78,82,NaN,R1004D Neuropsych.pdf,NaN,NaN,NaN,NaN,R1004D


Replace IQ score with the best available IQ score (see hierarchy bellow) and add nan if no BDI/BAI exist

In [63]:
iq = []
iq_type = []
dep = []
dep_type = []
anx = []
anx_type = []

for i in master.index:
    try:
        if pd.notnull(master.iloc[i]['FSIQ4']) == True:
            IQ = master.iloc[i]['FSIQ4']
            IQ_type = "FSIQ4"
        elif pd.notnull(master.iloc[i]['FSIQ3']) == True:
            IQ = master.iloc[i]['FSIQ3']
            IQ_type = "FSIQ3"
        elif pd.notnull(master.iloc[i]['FSIQ2 (Four Subtest)']) == True:
            IQ = master.iloc[i]['FSIQ2 (Four Subtest)']
            IQ_type = "4FSIQ2"
        elif pd.notnull(master.iloc[i]['FSIQ2 (Two Subtest)']) == True:
            IQ = master.iloc[i]['FSIQ2 (Two Subtest)']
            IQ_type = "2FSIQ2"
        elif pd.notnull(master.iloc[i]['FSIQ1 (Four Subtest)']) == True:
            IQ = master.iloc[i]['FSIQ1 (Four Subtest)']
            IQ_type = "4FSIQ1"
        elif pd.notnull(master.iloc[i]['FSIQ1 (Two Subtest)']) == True:
            IQ = master.iloc[i]['FSIQ1 (Two Subtest)']
            IQ_type = "2FSIQ1"
        elif pd.notnull(master.iloc[i]['GAI']) == True:
            IQ = master.iloc[i]['GAI']
            IQ_type = "GAI"
        elif pd.notnull(master.iloc[i]['VCI']) == True:
            IQ = master.iloc[i]['VCI']
            IQ_type = "VCI"
        elif pd.notnull(master.iloc[i]['PRI']) == True:
            IQ = master.iloc[i]['PRI']
            IQ_type = "PRI"
        elif pd.notnull(master.iloc[i]['WMI']) == True:
            IQ = master.iloc[i]['WMI']
            IQ_type = "WMI"
        else:
            IQ = np.nan
            IQ_type = np.nan
        iq.append(IQ)
        iq_type.append(IQ_type)

        if pd.notnull(master.iloc[i]['BDI']) == True:
            Dep = master.iloc[i]['BDI']
            Dep_type = "BDI"
        else:
            Dep = np.nan
            Dep_type = np.nan
        dep.append(Dep)
        dep_type.append(Dep_type)
        
        if pd.notnull(master.iloc[i]['BAI']) == True:
            Anx = master.iloc[i]['BAI']
            Anx_type = "BAI"
        else:
            Anx = np.nan
            Anx_type = np.nan
        anx.append(Anx)
        anx_type.append(Anx_type)

    except:
        print(i)

master["IQ type"] = iq_type
master["dep type"] = dep_type
master["anx type"] = anx_type
master['IQ'] = iq
master['depression'] = dep
master['anxiety'] = anx

###In master: Phase III implant refers to a reimplantation of a subject (rare)
master.head(4)

,Subject Number,Study site,Gender,Years of Education,Age of seizure onset,Age at Implant (calculated),Phase II Implant Date,Phase III Implant Date,Traumatic Brain Injury (TBI) Notes,Date of Pre-operative Report,BAI,BDI,PHQ-9,GAD-7,FSIQ4,FSIQ3,FSIQ2 (Four Subtest),FSIQ2 (Two Subtest),FSIQ1 (Four Subtest),FSIQ1 (Two Subtest),GAI,VCI,PRI,WMI,Pre-operative Report,Prior Neurosurgical Procedures (can select more than one),Prior Surgery: Resection Detail,Prior Surgery: Callosotomy Detail,Prior Surgery: Laser Ablation Detail,subject,IQ type,dep type,anx type,IQ,depression,anxiety
0,1,University of Pennsylvania,Female,13.0,20.0,48.105026,2014-10-08,NaT,NaN,NaT,8.0,2.0,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,76.0,81,75,NaN,REP001_neurospych.pdf,NaN,NaN,NaN,NaN,R1001P,FSIQ4,BDI,BAI,72,2.0,8.0
1,2,University of Pennsylvania,Female,16.0,34.0,49.383743,2014-11-12,NaT,NaN,NaT,9.0,0.0,NaN,NaN,86,NaN,NaN,NaN,NaN,NaN,84.0,89,84,NaN,R1002P_neuropsych.pdf,NaN,NaN,NaN,NaN,R1002P,FSIQ4,BDI,BAI,86,0.0,9.0
2,3,University of Pennsylvania,Female,13.0,37.0,39.390382,2014-11-13,NaT,motor vehicle accident due to loss of consciou...,NaT,NaN,NaN,NaN,NaN,102,NaN,NaN,NaN,NaN,NaN,NaN,107,100,NaN,R1003P_neuropsych.pdf,NaN,NaN,NaN,NaN,R1003P,FSIQ4,NaN,NaN,102,NaN,NaN
3,4,Dartmouth University,Female,12.0,6.0,51.918931,2014-11-06,NaT,"In early 90s, motor vehicle accident resulting...",NaT,NaN,NaN,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,78,82,NaN,R1004D Neuropsych.pdf,NaN,NaN,NaN,NaN,R1004D,FSIQ4,NaN,NaN,72,NaN,NaN


Check the available data in each of the collaborating sites

In [64]:
site_stats = pd.DataFrame([])
tot = 0

#Check how many subjects have all measures out of those that were tested and have a depression measure
for site in master['Study site'].unique():
    site_df = master[master['Study site']==site]
    tested = site_df.dropna(subset=['subject'])
    tot = tot + len(tested)
    site_stats=site_stats.append({
        'site':site,
        'total subs':len(site_df),
        'subs tested':len(tested),
        'gender':tested['Gender'].count(),
        'years of edu':tested['Years of Education'].count(),
        'age of sz onset':tested['Age of seizure onset'].count(),
        'age at implant':tested['Age at Implant (calculated)'].count(),
        'TBI':tested['Traumatic Brain Injury (TBI) Notes'].count(),
        'BAI':tested['BAI'].count(),
        'BDI':tested['BDI'].count(),
        'PHQ-9':tested['PHQ-9'].count(),
        'GAD-7':tested['GAD-7'].count(),
        'FSIQ4':tested['FSIQ4'].count(),
        'FSIQ3':tested['FSIQ3'].count(),
        '4FSIQ2':tested['FSIQ2 (Four Subtest)'].count(),
        '2FSIQ2':tested['FSIQ2 (Four Subtest)'].count(),
        '4FSIQ1':tested['FSIQ1 (Four Subtest)'].count(),
        '2FSIQ1':tested['FSIQ1 (Four Subtest)'].count(),
        'GAI':tested['GAI'].count(),
        'VCI':tested['VCI'].count(),
        'PRI':tested['PRI'].count(),
        'WMI':tested['WMI'].count(),
        'Consolidated IQ':tested['IQ'].count(),
        'Consolidated dep':tested['depression'].count(),
        'Consolidated anx':tested['anxiety'].count(),
        'Neurosurg procedure':tested['Prior Neurosurgical Procedures (can select more than one)'].count()
    }, ignore_index=True)
    
site_stats[['Consolidated IQ', 'Consolidated dep', 'Consolidated anx', 'site', 'subs tested','Neurosurg procedure','age of sz onset','age at implant','years of edu','gender']]
#site_stats

,Consolidated IQ,Consolidated dep,Consolidated anx,site,subs tested,Neurosurg procedure,age of sz onset,age at implant,years of edu,gender
0,27.0,20.0,9.0,University of Pennsylvania,31.0,6.0,31.0,31.0,15.0,31.0
1,44.0,41.0,33.0,Dartmouth University,52.0,10.0,50.0,51.0,44.0,52.0
2,68.0,74.0,73.0,Jefferson Hospital,79.0,31.0,78.0,79.0,56.0,79.0
3,2.0,0.0,0.0,University of Washington,2.0,0.0,2.0,2.0,2.0,2.0
4,79.0,40.0,38.0,Emory University,82.0,34.0,81.0,81.0,71.0,82.0
5,46.0,3.0,0.0,Mayo Clinic,58.0,15.0,57.0,58.0,52.0,58.0
6,70.0,12.0,6.0,UT Southwestern,107.0,24.0,98.0,107.0,82.0,107.0
7,19.0,1.0,1.0,Columbia University,27.0,5.0,26.0,27.0,19.0,27.0
8,23.0,6.0,3.0,NINDS,31.0,1.0,26.0,29.0,9.0,31.0


In [65]:
master.columns
# subset = master.dropna(subset=['subject'])
subset = master.dropna(subset=['IQ', 'depression', 'anxiety', 'Years of Education','Age at Implant (calculated)', 'Gender','Age of seizure onset'])
print("Number of subjects with all essential demographic/neuropsych measures = " + str(len(subset)))

Number of subjects with all essential demographic/neuropsych measures = 145


Create a dataframe with memory performance n a list (list_df) or session (trial_df) level

In [66]:
trial_df = pd.DataFrame([])
list_df = pd.DataFrame([])

experiments = ['FR1', 'catFR1']

for sub in master['subject']:
    try:
        for exp in experiments:
            localizations=df[(df['subject']==sub) & (df['experiment']==exp)]['localization'].unique()        

            for loc in localizations:
                montages = df[(df['subject']==sub) & (df['experiment']==exp) & (df['localization']==loc)]['montage'].unique()            

                for mont in montages: # iterate over montages            
                    data = df[np.logical_and(df["subject"] == sub, df['experiment']==exp)]
                    sessions=data[np.logical_and(data['montage']==mont,data['localization']==loc)]['session'].unique()

                    for sess in sessions: 
                        reader = CMLReader(sub, exp, sess, montage=mont, localization=loc)   #reader for loading CML data
                        evs = reader.load('task_events')
                        word_evs = evs[evs['type']=='REC_WORD']

                        #exclude practice lists 
                        lists = len(word_evs['list'][word_evs['list']>0].unique()) 
                        corr_recs = len(word_evs[word_evs['intrusion']==0])
                        ELIs = len(word_evs[word_evs['intrusion']==-1])
                        PLIs = len(word_evs[word_evs['intrusion']>0])
                        total_ints = ELIs + PLIs
                        total_trials = corr_recs + total_ints

                        trials = [] #make list of event numbers of analysis
                        for list_num in word_evs['list'][word_evs['list']>0].unique():
                            list_events=word_evs[word_evs['list']==list_num]
                            if len(list_events)>0: 
                                for i in range(0,len(list_events)):
                                    trials.append(list_events.index[i])
                            PLI = len(list_events[list_events['intrusion']>0])
                            ELI = len(list_events[list_events['intrusion']==-1])
                            correc = len(list_events[list_events['intrusion']==0])
                            list_df = list_df.append({ #create dataframe of events for analysis grouped by list ##should I filter out lists with no events?
                                'subject':sub,
                                'session': sess,
                                'experiment': exp,
                                'list': list_num,
                                'number of correct recalls': correc,
                                'number of extra-list intrusions': ELI,
                                'number of prior-list intrusions': PLI,
                                'total number of intrusions': ELI+PLI
                                },ignore_index=True)
                        trial_df = trial_df.append({ #create dataframe of events for analysis grouped by session
                            'subject':sub,
                            'session': sess,
                            'montage': mont,
                            'localization': loc,
                            'experiment': exp,
                            'number of lists': lists,
                            'number of correct recalls': corr_recs,
                            'number of extra-list intrusions': ELIs,
                            'number of prior-list intrusions': PLIs,
                            'total number of intrusions': total_ints,
                            'total number of trials left': total_trials,
                            'trials for analysis': trials
                            },ignore_index=True) 
    except:
        print('Failed for subject:', sub)
list_df[:100]

,experiment,list,number of correct recalls,number of extra-list intrusions,number of prior-list intrusions,session,subject,total number of intrusions
0,FR1,2.0,2.0,0.0,2.0,0.0,R1001P,2.0
1,FR1,3.0,4.0,3.0,1.0,0.0,R1001P,4.0
2,FR1,4.0,2.0,3.0,1.0,0.0,R1001P,4.0
3,FR1,7.0,2.0,2.0,0.0,0.0,R1001P,2.0
4,FR1,8.0,3.0,1.0,1.0,0.0,R1001P,2.0
...,...,...,...,...,...,...,...,...
95,FR1,1.0,5.0,3.0,0.0,0.0,R1003P,3.0
96,FR1,2.0,4.0,2.0,0.0,0.0,R1003P,2.0
97,FR1,3.0,5.0,0.0,0.0,0.0,R1003P,0.0
98,FR1,4.0,7.0,1.0,0.0,0.0,R1003P,1.0


Combine memory performance information with demographics, and neuropsych info

In [67]:
#BEGINNING OF SESSION LEVEL ANALYSES
df_master = pd.DataFrame([])
#Create master dataframe of important session stats (averaged by list), demographics, and neuropsych
for i in trial_df.index:
    try:
        sub = trial_df.iloc[i]['subject']
        eli = trial_df.iloc[i]['number of extra-list intrusions']
        pli = trial_df.iloc[i]['number of prior-list intrusions']
        correc = trial_df.iloc[i]['number of correct recalls']
        allrec = trial_df.iloc[i]['total number of trials left']
        lists = trial_df.iloc[i]['number of lists']
        session=trial_df.iloc[i]['session']
        exp = trial_df.iloc[i]['experiment']
        PLI_avg = float(pli/lists) #average per list
        ELI_avg = float(eli/lists)
        correc_avg = float(correc/lists)
        allrec_avg = float(allrec/lists)
        if allrec > -1: #or 0 to remove sessions with no valid recalls and no BAI or BDI
            df_master = df_master.append({
                'subject':sub,
                'experiment':exp,
                'correct recalls':correc,
                'ELIs':eli,
                'PLIs':pli,
                'total ints': eli+pli,
                'lists':lists,
                'session':session,
                'all recalls':allrec,
                'average_correct_recalls':correc_avg,
                'average_total_recalls':allrec_avg,
                'average_ELI':ELI_avg,
                'average_PLI':PLI_avg,
                'average_total_ints':ELI_avg+PLI_avg,
                'BAI':master[master['subject']==sub]['BAI'].item(),
                'BDI':master[master['subject']==sub]['BDI'].item(),
                'site':master[master['subject']==sub]['Study site'].item(),
                'gender':master[master['subject']==sub]['Gender'].item(),
                'years_of_education':master[master['subject']==sub]['Years of Education'].item(),
                'age_of_seizure_onset':master[master['subject']==sub]['Age of seizure onset'].item(),
                'age_at_implant':master[master['subject']==sub]['Age at Implant (calculated)'].item(),
                'implant_date':master[master['subject']==sub]['Phase II Implant Date'].item(),
                'reimplant_date':master[master['subject']==sub]['Phase III Implant Date '].item(),
                'TBI_notes':master[master['subject']==sub]['Traumatic Brain Injury (TBI) Notes'].item(),
                'binary_TBI':pd.isnull(master[master['subject']==sub]['Traumatic Brain Injury (TBI) Notes'].item()),
                'date_of_neuropsych_report':master[master['subject']==sub]['Date of Pre-operative Report'].item(),
                'IQ':master[master['subject']==sub]['IQ'].item(),
                'depression':master[master['subject']==sub]['depression'].item(),
                'anxiety':master[master['subject']==sub]['anxiety'].item(),
                'IQ_type':master[master['subject']==sub]['IQ type'].item(),
                'dep_type':master[master['subject']==sub]['dep type'].item(),
                'anx_type':master[master['subject']==sub]['anx type'].item(),
#                 'binary_prior_neuro':binary_neuro,
                'binary_prior_neuro': pd.isnull(master[master['subject']==sub]['Prior Neurosurgical Procedures (can select more than one)'].item()), 
                'prior_neurosurgical_procedures':master[master['subject']==sub]['Prior Neurosurgical Procedures (can select more than one)'].item(),    
            }, ignore_index = True)
        else:
            print('removed row =', i, sub, exp, 'due to no valid recalls')
    except:
        print('failed for row =', i, sub, exp)
# df_master = df_master.dropna(subset=['BAI', 'BDI'], how='all') #drop sessions from subjects that have neither score

### In df_master: Phase III implant refers to a reimplantation of a subject (rare), 
### binary_TBI = 1 when sub did NOT have TBI notes, binary_prior_neuro = 1 when sub did NOT have prior neurosurg notes
df_master[:10]


/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()
/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars


,BAI,BDI,ELIs,IQ,IQ_type,PLIs,TBI_notes,age_at_implant,age_of_seizure_onset,all recalls,anx_type,anxiety,average_ELI,average_PLI,average_correct_recalls,average_total_ints,average_total_recalls,binary_TBI,binary_prior_neuro,correct recalls,date_of_neuropsych_report,dep_type,depression,experiment,gender,implant_date,lists,prior_neurosurgical_procedures,reimplant_date,session,site,subject,total ints,years_of_education
0,8.0,2.0,21.0,72.0,FSIQ4,25.0,NaN,48.105026,20.0,108.0,BAI,8.0,0.954545,1.136364,2.818182,2.090909,4.909091,1.0,1.0,62.0,NaT,BDI,2.0,FR1,Female,2014-10-08,22.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,46.0,13.0
1,8.0,2.0,19.0,72.0,FSIQ4,17.0,NaN,48.105026,20.0,99.0,BAI,8.0,0.826087,0.739130,2.739130,1.565217,4.304348,1.0,1.0,63.0,NaT,BDI,2.0,FR1,Female,2014-10-08,23.0,NaN,NaT,1.0,University of Pennsylvania,R1001P,36.0,13.0
2,9.0,0.0,3.0,86.0,FSIQ4,8.0,NaN,49.383743,34.0,128.0,BAI,9.0,0.120000,0.320000,4.680000,0.440000,5.120000,1.0,1.0,117.0,NaT,BDI,0.0,FR1,Female,2014-11-12,25.0,NaN,NaT,0.0,University of Pennsylvania,R1002P,11.0,16.0
3,9.0,0.0,2.0,86.0,FSIQ4,3.0,NaN,49.383743,34.0,125.0,BAI,9.0,0.080000,0.120000,4.800000,0.200000,5.000000,1.0,1.0,120.0,NaT,BDI,0.0,FR1,Female,2014-11-12,25.0,NaN,NaT,1.0,University of Pennsylvania,R1002P,5.0,16.0
4,NaN,NaN,30.0,102.0,FSIQ4,26.0,motor vehicle accident due to loss of consciou...,39.390382,37.0,142.0,NaN,NaN,1.363636,1.181818,3.909091,2.545455,6.454545,0.0,1.0,86.0,NaT,NaN,NaN,FR1,Female,2014-11-13,22.0,NaN,NaT,0.0,University of Pennsylvania,R1003P,56.0,13.0
5,NaN,NaN,27.0,102.0,FSIQ4,47.0,motor vehicle accident due to loss of consciou...,39.390382,37.0,185.0,NaN,NaN,1.080000,1.880000,4.440000,2.960000,7.400000,0.0,1.0,111.0,NaT,NaN,NaN,FR1,Female,2014-11-13,25.0,NaN,NaT,1.0,University of Pennsylvania,R1003P,74.0,13.0
6,NaN,NaN,3.0,72.0,FSIQ4,6.0,"In early 90s, motor vehicle accident resulting...",51.918931,6.0,24.0,NaN,NaN,0.272727,0.545455,1.363636,0.818182,2.181818,0.0,1.0,15.0,NaT,NaN,NaN,catFR1,Female,2014-11-06,11.0,NaN,NaT,0.0,Dartmouth University,R1004D,9.0,12.0
7,7.0,7.0,9.0,83.0,FSIQ4,21.0,NaN,20.553582,18.0,136.0,BAI,7.0,0.360000,0.840000,4.240000,1.200000,5.440000,1.0,1.0,106.0,NaT,BDI,7.0,FR1,Female,2015-01-28,25.0,NaN,NaT,0.0,University of Pennsylvania,R1006P,30.0,14.0
8,7.0,7.0,7.0,83.0,FSIQ4,3.0,NaN,20.553582,18.0,62.0,BAI,7.0,0.538462,0.230769,4.000000,0.769231,4.769231,1.0,1.0,52.0,NaT,BDI,7.0,FR1,Female,2015-01-28,13.0,NaN,NaT,1.0,University of Pennsylvania,R1006P,10.0,14.0
9,2.0,1.0,2.0,92.0,GAI,12.0,NaN,30.801454,22.0,70.0,BAI,2.0,0.086957,0.521739,2.434783,0.608696,3.043478,1.0,1.0,56.0,2012-02-26,BDI,1.0,FR1,Female,2014-12-02,23.0,NaN,NaT,0.0,Jefferson Hospital,R1010J,14.0,16.0


In [9]:
# save master df
with open('/home1/noaherz/Mental_health5/results/df_master','wb') as filehandle:
    pickle.dump(df_master,filehandle)



### Load the dataframe

In [99]:
# load
with open('/home1/noaherz/Mental_health5/results/df_master','rb') as pickle_file:
    df_master=pickle.load(pickle_file)


In [7]:
# Noa_subs = df_master.dropna(subset=['IQ', 'depression', 'anxiety', 'years_of_education','age_at_implant', 'gender','age_of_seizure_onset'])
# %store Noa_subs
# save Noa_subs
# Noa_subs.to_pickle('/home1/noaherz/Mental_health/Noa_subs')
# print("Number of subjects with all essential demographic/neuropsych measures = " + str(len(subset['subject'].unique())))


The next cell creates the same df, but this time on a list level. You can skip this part.

In [68]:
#BEGINNING OF LIST LEVEL ANALYSES: THIS CELL TAKES A COUPLE OF MINUTES TO PARSE THROUGH ALL LISTS! Better way to do this? @Noa
df_list_master = pd.DataFrame([])
#Create master dataframe of important LIST stats, demographics, and neuropsych
for i in list_df.index:
    try:
        sub = list_df.iloc[i]['subject']
        eli = list_df.iloc[i]['number of extra-list intrusions']
        pli = list_df.iloc[i]['number of prior-list intrusions']
        correc = list_df.iloc[i]['number of correct recalls']
        allrec = eli+pli+correc
        listnum = list_df.iloc[i]['list']
        session = list_df.iloc[i]['session']
        exp = list_df.iloc[i]['experiment']
        
#         if allrec > 0: #remove sessions with no valid recalls
        df_list_master = df_list_master.append({
            'subject':sub,
            'experiment':exp,
            'list':listnum,
            'session':session,
            'correct_recalls':correc,
            'ELIs':eli,
            'PLIs':pli,
            'total_ints': eli+pli,
            'all_recalls':allrec,
            'BAI':master[master['subject']==sub]['BAI'].item(),
            'BDI':master[master['subject']==sub]['BDI'].item(),
            'site':master[master['subject']==sub]['Study site'].item(),
            'gender':master[master['subject']==sub]['Gender'].item(),
            'years_of_education':master[master['subject']==sub]['Years of Education'].item(),
            'age_of_seizure_onset':master[master['subject']==sub]['Age of seizure onset'].item(),
            'age_at_implant':master[master['subject']==sub]['Age at Implant (calculated)'].item(),
            'implant_date':master[master['subject']==sub]['Phase II Implant Date'].item(),
            'reimplant_date':master[master['subject']==sub]['Phase III Implant Date '].item(),
            'TBI_notes':master[master['subject']==sub]['Traumatic Brain Injury (TBI) Notes'].item(),
            'binary_TBI':pd.isnull(master[master['subject']==sub]['Traumatic Brain Injury (TBI) Notes'].item()),
            'date_of_neuropsych_report':master[master['subject']==sub]['Date of Pre-operative Report'].item(),
            'IQ':master[master['subject']==sub]['IQ'].item(),
            'depression':master[master['subject']==sub]['depression'].item(),
            'anxiety':master[master['subject']==sub]['anxiety'].item(),
            'IQ_type':master[master['subject']==sub]['IQ type'].item(),
            'dep_type':master[master['subject']==sub]['dep type'].item(),
            'anx_type':master[master['subject']==sub]['anx type'].item(),
            'binary_prior_neuro': pd.isnull(master[master['subject']==sub]['Prior Neurosurgical Procedures (can select more than one)'].item()), 
            'prior_neurosurgical_procedures':master[master['subject']==sub]['Prior Neurosurgical Procedures (can select more than one)'].item(),    
        }, ignore_index = True)
#         else:
#             print('removed row =', i, sub, exp, 'due to no valid recalls')
    except:
        print('failed for row =', i, sub, exp)

### In df_master: Phase III implant refers to a reimplantation of a subject (rare), 
### binary_TBI = 1 when sub did NOT have TBI notes, binary_prior_neuro = 1 when sub did NOT have prior neurosurg notes
df_list_master[:10]

,BAI,BDI,ELIs,IQ,IQ_type,PLIs,TBI_notes,age_at_implant,age_of_seizure_onset,all_recalls,anx_type,anxiety,binary_TBI,binary_prior_neuro,correct_recalls,date_of_neuropsych_report,dep_type,depression,experiment,gender,implant_date,list,prior_neurosurgical_procedures,reimplant_date,session,site,subject,total_ints,years_of_education
0,8.0,2.0,0.0,72.0,FSIQ4,2.0,NaN,48.105026,20.0,4.0,BAI,8.0,1.0,1.0,2.0,NaT,BDI,2.0,FR1,Female,2014-10-08,2.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,2.0,13.0
1,8.0,2.0,3.0,72.0,FSIQ4,1.0,NaN,48.105026,20.0,8.0,BAI,8.0,1.0,1.0,4.0,NaT,BDI,2.0,FR1,Female,2014-10-08,3.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,4.0,13.0
2,8.0,2.0,3.0,72.0,FSIQ4,1.0,NaN,48.105026,20.0,6.0,BAI,8.0,1.0,1.0,2.0,NaT,BDI,2.0,FR1,Female,2014-10-08,4.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,4.0,13.0
3,8.0,2.0,2.0,72.0,FSIQ4,0.0,NaN,48.105026,20.0,4.0,BAI,8.0,1.0,1.0,2.0,NaT,BDI,2.0,FR1,Female,2014-10-08,7.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,2.0,13.0
4,8.0,2.0,1.0,72.0,FSIQ4,1.0,NaN,48.105026,20.0,5.0,BAI,8.0,1.0,1.0,3.0,NaT,BDI,2.0,FR1,Female,2014-10-08,8.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,2.0,13.0
5,8.0,2.0,2.0,72.0,FSIQ4,2.0,NaN,48.105026,20.0,6.0,BAI,8.0,1.0,1.0,2.0,NaT,BDI,2.0,FR1,Female,2014-10-08,9.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,4.0,13.0
6,8.0,2.0,3.0,72.0,FSIQ4,2.0,NaN,48.105026,20.0,5.0,BAI,8.0,1.0,1.0,0.0,NaT,BDI,2.0,FR1,Female,2014-10-08,10.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,5.0,13.0
7,8.0,2.0,0.0,72.0,FSIQ4,0.0,NaN,48.105026,20.0,2.0,BAI,8.0,1.0,1.0,2.0,NaT,BDI,2.0,FR1,Female,2014-10-08,11.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,0.0,13.0
8,8.0,2.0,1.0,72.0,FSIQ4,1.0,NaN,48.105026,20.0,3.0,BAI,8.0,1.0,1.0,1.0,NaT,BDI,2.0,FR1,Female,2014-10-08,12.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,2.0,13.0
9,8.0,2.0,1.0,72.0,FSIQ4,3.0,NaN,48.105026,20.0,6.0,BAI,8.0,1.0,1.0,2.0,NaT,BDI,2.0,FR1,Female,2014-10-08,13.0,NaN,NaT,0.0,University of Pennsylvania,R1001P,4.0,13.0


In [9]:
# #Checking number of subjects included from different sites
# sub_ar = df_master['subject'].unique()
# res = sum(1 for i in sub_ar if "J" in i)
# # sub_ar
# res

 ## Linear mixed effects model

The bellow statistics checks for experiment main effect (catFR/FR) on the total number of intrusions (not sig) and correct-recllas (catFR produce more correct recalls compared to FR)


In [100]:
df_master=df_master.dropna(axis='index',subset=['average_total_ints'])
any(np.isnan(df_master['average_total_ints']))

False

#### Check main effect of experiment on intrusions and correct recalls

In [101]:

vc = {'session': '0 + C(session)'} #the vc formula always varies within the groups parameter, so it will vary within subject 
# If we want to use list as another nested variable, we can use this:
# vc = {'session': '0 + C(session)', 'list': '0 + C(list):C(session)'}

## Total number of intrusions:
model = sm.MixedLM.from_formula('average_total_ints ~ 1',vc_formula=vc,  groups='subject', data=df_master) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# Experiment main effect
model = sm.MixedLM.from_formula('average_total_ints ~ C(experiment)',vc_formula=vc, groups='subject', data=df_master) 
result=model.fit(reml=False, method='nm')
llf_exp =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of experiment on total intrusions')
lr, p = lrtest(llf_empty, llf_exp, df = 1) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

## Total number of correct-recalls:
model = sm.MixedLM.from_formula('average_correct_recalls ~ 1',vc_formula=vc,  groups='subject', data=df_master) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# Experiment main effect
model = sm.MixedLM.from_formula('average_correct_recalls ~ C(experiment)',vc_formula=vc, groups='subject', data=df_master) 
result=model.fit(reml=False, method='nm')
llf_exp =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects of experiment on correct-recalls
print('Main effect of experiment on correct-recalls')
lr, p = lrtest(llf_empty, llf_exp, df = 1) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))



             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: average_total_ints
No. Observations: 1103    Method:             ML                
No. Groups:       365     Scale:              0.2019            
Min. group size:  1       Log-Likelihood:     -1342.9274        
Max. group size:  13      Converged:          Yes               
Mean group size:  3.0                                           
------------------------------------------------------------------
               Coef.   Std.Err.     z      P>|z|   [0.025   0.975]
------------------------------------------------------------------
Intercept      0.977      0.028   34.386   0.000    0.921    1.033
session Var    0.561      0.188                                   

              Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: average_total_ints
No. Observations:   1103    Method:             ML                
No. Groups:         365     Scale:   

Let's start with a simple model that predicts intrusions/correct-reclls based on BAI+BDI scores (without other factors)
#### check influence of BDI and BAI beyond experiment

In [102]:
## Total number of intrusions:

df_BAI_BDI_reduced=df_master.dropna(axis='index',subset=['BDI','BAI'])

model = sm.MixedLM.from_formula('average_total_ints ~ C(experiment)',vc_formula=vc,  groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# BDI+BAI variables - main effect
model = sm.MixedLM.from_formula('average_total_ints ~ BDI+BAI+C(experiment)',vc_formula=vc, groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of BAI+BDI on total intrusions')
lr, p = lrtest(llf_empty, llf_demog, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# BDIvariables - main effect
model = sm.MixedLM.from_formula('average_total_ints ~ BDI+C(experiment)',vc_formula=vc, groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_BDI =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of BDI on total intrusions')
lr, p = lrtest(llf_empty, llf_BDI, df = 1) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# BDIvariables - main effect
model = sm.MixedLM.from_formula('average_total_ints ~ BAI+C(experiment)',vc_formula=vc, groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_BAI =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of BAI on total intrusions')
lr, p = lrtest(llf_empty, llf_BAI, df = 1) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

## Total number of correct-recalls:

model = sm.MixedLM.from_formula('average_correct_recalls ~ C(experiment)',vc_formula=vc,  groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# BDI+BAI variables - main effect
model = sm.MixedLM.from_formula('average_correct_recalls ~ BDI+BAI+C(experiment)',vc_formula=vc, groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of BAI+BDI on total correct recalls:')
lr, p = lrtest(llf_empty, llf_demog, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# BDIvariables - main effect
model = sm.MixedLM.from_formula('average_correct_recalls ~ BDI+C(experiment)',vc_formula=vc, groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_BDI =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of BDI on average_correct_recalls')
lr, p = lrtest(llf_empty, llf_BDI, df = 1) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# BDIvariables - main effect
model = sm.MixedLM.from_formula('average_correct_recalls ~ BAI+C(experiment)',vc_formula=vc, groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_BAI =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of BAI on average_correct_recalls')
lr, p = lrtest(llf_empty, llf_BAI, df = 1) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

              Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: average_total_ints
No. Observations:   401     Method:             ML                
No. Groups:         120     Scale:              0.2129            
Min. group size:    1       Log-Likelihood:     -482.7256         
Max. group size:    10      Converged:          Yes               
Mean group size:    3.3                                           
------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept                1.081    0.056 19.395 0.000  0.972  1.190
C(experiment)[T.catFR1] -0.159    0.065 -2.454 0.014 -0.286 -0.032
session Var              0.511    0.304                           

              Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: average_total_ints
No. Observations:   401 

These results indicate that:
Higher BAI scores are associated with both more intrusions and more correct-reclls (suggesting that they have a more liberal threshorld for recalling words...?)
Higher BDI scores are associated with less correct recalls

Overall, BAI and BDI scores are significant predictors of the number of correct-recalls (trending significance for the number of intrusions)

We can also predict PLIs/ELIs based on BDI+BAI score:
#### chack influence of BDI and BAI on PLI/ELI, beyond experiment    

In [103]:
##  number of PL intrusions:


df_BAI_BDI_reduced=df_master.dropna(axis='index',subset=['BDI','BAI'])

model = sm.MixedLM.from_formula('average_PLI ~ C(experiment)',vc_formula=vc,  groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# BDI+BAI variables - main effect
model = sm.MixedLM.from_formula('average_PLI ~ BAI+BDI+C(experiment)',vc_formula=vc, groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_recall =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of BAI+BDI on average_PLI')
lr, p = lrtest(llf_empty, llf_recall, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

##  number of EL intrusions:

model = sm.MixedLM.from_formula('average_ELI ~ C(experiment)',vc_formula=vc,  groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# BDI+BAI variables - main effect
model = sm.MixedLM.from_formula('average_ELI ~ BAI+BDI+C(experiment)',vc_formula=vc, groups='subject', data=df_BAI_BDI_reduced) 
result=model.fit(reml=False, method='nm')
llf_recall =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of BAI+BDI on average_ELI')
lr, p = lrtest(llf_empty, llf_recall, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))


              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    average_PLI
No. Observations:    401        Method:                ML         
No. Groups:          120        Scale:                 0.0917     
Min. group size:     1          Log-Likelihood:        -267.8060  
Max. group size:     10         Converged:             Yes        
Mean group size:     3.3                                          
------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept                0.518    0.033 15.658 0.000  0.453  0.583
C(experiment)[T.catFR1] -0.002    0.041 -0.041 0.967 -0.082  0.079
session Var              0.148    0.158                           

              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    average_PLI
No. Observations:    401

These results suggest that:
    Higher BAI scores are associated specifically with more PLIs (and not ELIs)

Here I predict the rate of intrusions/correct-recalls, based on: sex, prior resection, years of education, age at seizure onset, age of the patient, and general intelligence score (We can change these factors, but I chose here the ones that seemed the most relevant).

I then compare this model to a model that also includes BDI+BAI scores, to see whether those contribute to intrusions prediction/correct-recalls beyond these factors.

Note: the model cannot accept nan values. For each variable that has nan, we are loosing the whole subject, so for these variables let's try and maximize the data we have available. Currently only n=148 have the full data (with all demographic variables)

In [104]:
len(df_BAI_BDI_reduced['subject'].unique())
# df_master_reduced['prior_neurosurgical_procedures']

120

### Choose demographics to include in the model:

In [106]:
# # demographic factors main effect on the total number of intrusions and correct-recllas 
list_demographics_columns=['experiment','gender','age_at_implant','IQ','anxiety','depression','age_of_seizure_onset','binary_prior_neuro'] # other anxiety and depression scores need to be normalized
number_of_demog_factors=len(list_demographics_columns)-2
# remove nan's from df:
df_master_reduced=df_master.dropna(axis='index',subset=list_demographics_columns)


In [74]:
# select_subs = list(df_master_reduced['subject'].unique())
# select_subs
# with open("select_subs.txt", "w") as output:
#     output.write(str(select_subs))

#### Check the effects of depression/anxiety on total intrusions, beyond demographic variables

In [107]:
vc = {'session': '0 + C(session)'} #the vc formula always varies within the groups parameter, so it will vary within subject 

## Total number of intrusions:
model = sm.MixedLM.from_formula('average_total_ints ~ 1',vc_formula=vc,  groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# demographis variables - main effect
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any kind of procedure WAS present 
model = sm.MixedLM.from_formula('average_total_ints ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ',vc_formula=vc, groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of demographics on total intrusions')
lr, p = lrtest(llf_empty, llf_demog, df = number_of_demog_factors) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# demographis variables + depression - main effect ()
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('average_total_ints ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ + depression + anxiety',vc_formula=vc, groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog_Beck =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for depression
print('Main effect of depression and anxiety on total intrusions, beyond other demographic variables:')
lr, p = lrtest(llf_demog, llf_demog_Beck, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))



             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: average_total_ints
No. Observations: 364     Method:             ML                
No. Groups:       109     Scale:              0.2394            
Min. group size:  1       Log-Likelihood:     -443.9402         
Max. group size:  10      Converged:          Yes               
Mean group size:  3.3                                           
------------------------------------------------------------------
               Coef.   Std.Err.     z      P>|z|   [0.025   0.975]
------------------------------------------------------------------
Intercept      0.990      0.048   20.443   0.000    0.895    1.085
session Var    0.502      0.299                                   

                 Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   average_total_ints
No. Observations:    364       Method:               ML                
No. Groups:          109

#### check influence of depression and anxiery, beyond demographics, on PLIs

In [109]:
#Check effect of depression and anxiety separately on PLIs

# Total number of intrusions
# remove nan's from df:
# df_master_reduced=df_master.dropna(axis='index',subset=list_demographics_columns)

## Total number of intrusions:
model = sm.MixedLM.from_formula('average_PLI ~ 1',vc_formula=vc,  groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# demographis variables - main effect
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('average_PLI ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ',vc_formula=vc, groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of demographics on PLIs')
lr, p = lrtest(llf_empty, llf_demog, df = number_of_demog_factors) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# demographis variables + depression - main effect ()
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('average_PLI ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ + depression + anxiety',vc_formula=vc, groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog_Beck =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for depression
print('Main effect of depression on PLIs, beyond other demographic variables:')
lr, p = lrtest(llf_demog, llf_demog_Beck, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

## ELIs

## Total number of intrusions:
model = sm.MixedLM.from_formula('average_ELI ~ 1',vc_formula=vc,  groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# demographis variables - main effect
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('average_ELI ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ',vc_formula=vc, groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of demographics on ELIs')
lr, p = lrtest(llf_empty, llf_demog, df = number_of_demog_factors) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# demographis variables + depression - main effect ()
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('average_ELI ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ + depression + anxiety',vc_formula=vc, groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog_Beck =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for depression
print('Main effect of depression on ELIs, beyond other demographic variables:')
lr, p = lrtest(llf_demog, llf_demog_Beck, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))




          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: average_PLI
No. Observations: 364     Method:             ML         
No. Groups:       109     Scale:              0.0899     
Min. group size:  1       Log-Likelihood:     -240.7816  
Max. group size:  10      Converged:          Yes        
Mean group size:  3.3                                    
---------------------------------------------------------
                Coef. Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept       0.507    0.027 18.569 0.000  0.454  0.561
session Var     0.148    0.158                           

                 Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     average_PLI
No. Observations:       364         Method:                 ML         
No. Groups:             109         Scale:                  0.0967     
Min. group size:        1           Log-Li

#### Check effect of anxiety and depression on correct recalls 


In [111]:

## Total number of recalls:
model = sm.MixedLM.from_formula('average_correct_recalls ~ 1',vc_formula=vc,  groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# demographis variables - main effect
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('average_correct_recalls ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ',vc_formula=vc, groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of demographics on correct recalls')
lr, p = lrtest(llf_empty, llf_demog, df = 6) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# demographic variables + depression - main effect ()
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('average_correct_recalls ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ + depression + anxiety',vc_formula=vc, groups='subject', data=df_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog_Beck =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for depression
print('Main effect of depression on correct recalls, beyond other demographic variables:')
lr, p = lrtest(llf_demog, llf_demog_Beck, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))



                Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: average_correct_recalls
No. Observations: 364     Method:             ML                     
No. Groups:       109     Scale:              2.5062                 
Min. group size:  1       Log-Likelihood:     -728.5992              
Max. group size:  10      Converged:          Yes                    
Mean group size:  3.3                                                
------------------------------------------------------------------------
                Coef.    Std.Err.      z       P>|z|    [0.025    0.975]
------------------------------------------------------------------------
Intercept       3.950       0.097    40.631    0.000     3.759     4.140
session Var     0.728       0.388                                       

                 Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: average_correct_recalls
No. Observations:   364     Metho

### list-level lmm

In [78]:
# # Looking at effect of depression and anxiety on correct recalls within session/list model

# vc_list = {'session': '0 + C(session)', 'list': '0 + C(list):C(session)'}

# # Total number of intrusions
# # remove nan's from df:
# df_list_master_reduced=df_list_master.dropna(axis='index',subset=list_demographics_columns)

# ## Total number of intrusions:
# model = sm.MixedLM.from_formula('total_ints ~ 1',vc_formula=vc_list,  groups='subject', data=df_list_master_reduced) 
# result=model.fit(reml=False, method='nm')
# llf_empty =result.llf #log likelihood
# print(result.summary())

# # demographis variables - main effect
# # binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
# model = sm.MixedLM.from_formula('total_ints ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ',vc_formula=vc_list, groups='subject', data=df_list_master_reduced) 
# result=model.fit(reml=False, method='nm')
# llf_demog =result.llf #log likelihood
# print(result.summary())

# # Likelihood ratio tests for main effects
# print('Main effect of demographics on total intrusions')
# lr, p = lrtest(llf_empty, llf_demog, df = 6) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
# print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# # demographis variables + depression + anxiety - main effect ()
# # binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
# model = sm.MixedLM.from_formula('total_ints ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ + depression + anxiety',vc_formula=vc_list, groups='subject', data=df_list_master_reduced) 
# result=model.fit(reml=False, method='nm')
# llf_demog_Beck =result.llf #log likelihood
# print(result.summary())

# # Likelihood ratio tests for anxiety
# print('Main effect of BDI/BAI on total intrusions, beyond other demographic variables:')
# lr, p = lrtest(llf_demog, llf_demog_Beck, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
# print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))


# ## Total number of intrusions:
# model = sm.MixedLM.from_formula('correct_recalls ~ 1',vc_formula=vc_list,  groups='subject', data=df_list_master_reduced) 
# result=model.fit(reml=False, method='nm')
# llf_empty =result.llf #log likelihood
# print(result.summary())

# # demographis variables - main effect
# # binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
# model = sm.MixedLM.from_formula('correct_recalls ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ',vc_formula=vc_list, groups='subject', data=df_list_master_reduced) 
# result=model.fit(reml=False, method='nm')
# llf_demog =result.llf #log likelihood
# print(result.summary())

# # Likelihood ratio tests for main effects
# print('Main effect of demographics on correct recalls')
# lr, p = lrtest(llf_empty, llf_demog, df = 5) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
# print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# # demographis variables + depression + anxiety - main effect ()
# # binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
# model = sm.MixedLM.from_formula('correct_recalls ~ C(experiment)+C(gender)+C(binary_prior_neuro) + age_at_implant + age_of_seizure_onset + IQ + depression + anxiety',vc_formula=vc_list, groups='subject', data=df_list_master_reduced) 
# result=model.fit(reml=False, method='nm')
# llf_demog_Beck =result.llf #log likelihood
# print(result.summary())

# # Likelihood ratio tests for anxiety
# print('Main effect of BDI/BAI on correct recalls, beyond other demographic variables:')
# lr, p = lrtest(llf_demog, llf_demog_Beck, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
# print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: total_ints
No. Observations: 6218    Method:             ML        
No. Groups:       109     Scale:              0.9256    
Min. group size:  5       Log-Likelihood:     -8982.2962
Max. group size:  248     Converged:          Yes       
Mean group size:  57.0                                  
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.969    0.048 20.009 0.000  0.874  1.063
list Var       0.000    0.037                           
session Var    0.656    0.066                           



/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                 Mixed Linear Model Regression Results
Model:                 MixedLM      Dependent Variable:      total_ints
No. Observations:      6218         Method:                  ML        
No. Groups:            109          Scale:                   0.9234    
Min. group size:       5            Log-Likelihood:          -8949.9111
Max. group size:       248          Converged:               Yes       
Mean group size:       57.0                                            
-----------------------------------------------------------------------
                             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------
Intercept                     0.593    0.382  1.551 0.121 -0.156  1.341
C(experiment)[T.catFR1]      -0.140    0.039 -3.588 0.000 -0.217 -0.064
C(gender)[T.Male]             0.126    0.093  1.356 0.175 -0.056  0.308
C(binary_prior_neuro)[T.1.0]  0.055    0.111  0.493 0.622 -0.162  0.272
age_at_im

/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


...

/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


            Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: correct_recalls
No. Observations: 6218    Method:             ML             
No. Groups:       109     Scale:              3.1907         
Min. group size:  5       Log-Likelihood:     -12832.9221    
Max. group size:  248     Converged:          Yes            
Mean group size:  57.0                                       
--------------------------------------------------------------
                Coef.  Std.Err.    z     P>|z|  [0.025  0.975]
--------------------------------------------------------------
Intercept       3.772     0.091  41.529  0.000   3.594   3.950
list Var        0.000     0.065                               
session Var     2.314     0.126                               



/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                 Mixed Linear Model Regression Results
Model:                MixedLM    Dependent Variable:    correct_recalls
No. Observations:     6218       Method:                ML             
No. Groups:           109        Scale:                 3.0613         
Min. group size:      5          Log-Likelihood:        -12656.4711    
Max. group size:      248        Converged:             Yes            
Mean group size:      57.0                                             
-----------------------------------------------------------------------
                             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------
Intercept                    -0.002    0.652 -0.003 0.998 -1.280  1.276
C(experiment)[T.catFR1]       1.180    0.070 16.743 0.000  1.042  1.318
C(gender)[T.Male]             0.201    0.159  1.269 0.204 -0.110  0.512
C(binary_prior_neuro)[T.1.0] -0.415    0.189 -2.198 0.028 -0.786 -0.045
age_at_im

/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


This belongs in the emotionality notebook:

In [74]:
# Looking at effect of depression and anxiety on correct recalls within session/list model

vc_list = {'session': '0 + C(session)'}

#Check effect of list, anxiety, and depression on intrusions
# demographics_columns=['years_of_education','age_at_implant','IQ','anxiety','depression'] 

# Total number of intrusions
# remove nan's from df:
df_list_master_reduced=df_list_master.dropna(axis='index',subset=demographics_columns)

## Total number of intrusions:
model = sm.MixedLM.from_formula('avg_valence ~ 1',vc_formula=vc_list,  groups='subject', data=df_list_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# demographis variables - main effect
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('avg_valence ~ anxiety+depression',vc_formula=vc_list, groups='subject', data=df_list_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_emot =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of anx/dep on valence of total intrusions')
lr, p = lrtest(llf_empty, llf_emot, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# demographis variables - main effect
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('avg_valence ~ C(gender)+C(binary_prior_neuro)+ years_of_education + age_at_implant + IQ',vc_formula=vc_list, groups='subject', data=df_list_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of demographics on average valence of intrusions')
lr, p = lrtest(llf_empty, llf_demog, df = 5) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# demographis variables + depression + anxiety - main effect ()
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('total_ints ~ C(gender)+C(binary_prior_neuro)+ years_of_education + age_at_implant + IQ + depression + anxiety',vc_formula=vc_list, groups='subject', data=df_list_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog_Beck =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for anxiety
print('Main effect of BDI/BAI on total intrusions, beyond other demographic variables:')
lr, p = lrtest(llf_demog, llf_demog_Beck, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))


## Total number of intrusions:
model = sm.MixedLM.from_formula('correct_recalls ~ 1',vc_formula=vc_list,  groups='subject', data=df_list_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_empty =result.llf #log likelihood
print(result.summary())

# demographis variables - main effect
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('correct_recalls ~ C(gender)+C(binary_prior_neuro)+ years_of_education + age_at_implant + IQ',vc_formula=vc_list, groups='subject', data=df_list_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for main effects
print('Main effect of demographics on correct recalls')
lr, p = lrtest(llf_empty, llf_demog, df = 5) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

# demographis variables + depression + anxiety - main effect ()
# binary_prior_neuro is 'True' when prior neuro procedure was NOT present, 'False' when any king of procedure WAS present 
model = sm.MixedLM.from_formula('correct_recalls ~ C(gender)+C(binary_prior_neuro)+ years_of_education + age_at_implant + IQ + depression + anxiety',vc_formula=vc_list, groups='subject', data=df_list_master_reduced) 
result=model.fit(reml=False, method='nm')
llf_demog_Beck =result.llf #log likelihood
print(result.summary())

# Likelihood ratio tests for anxiety
print('Main effect of BDI/BAI on correct recalls, beyond other demographic variables:')
lr, p = lrtest(llf_demog, llf_demog_Beck, df = 2) # Degrees of freedom = number of regressors (i.e. columns in summary table) added when including the effect of interest, for categorical variable with two groups this is df = 1
print('LR test, p value: {:.4f}, {:.30f}'.format(lr, p))

PatsyError: Error evaluating factor: NameError: name 'avg_valence' is not defined
    avg_valence ~ 1
    ^^^^^^^^^^^